In [ ]:
from ultralytics import YOLO
import torch

In [ ]:
"""
This Notebook will be tha base for all the YOLO training that Minna does in the project.
"""

In [ ]:
if torch.cuda.is_available():
    device = 0
    print(f"Training on GPU")
else:
    print(f"Training on CPU")
    device = "cpu"

In [ ]:
!nvidia-smi | grep MiB

In [ ]:
"""
Default values used for all YOLO trainings.

These parameters should remain consistent across different training runs,
unless the hardware or training context changes significantly.
"""



model = YOLO("yolo11s.pt")  # pretrained YOLOv11L model

default_args = {
    "data": "config/materials.yaml",     # dataset configuration path
    "project": "material_koulutus",      # output base folder
    "imgsz": 850,                     # default input size for YOLO
    "batch": 8,                      # maximum batch size (GPU memory limit)
    "device": device,                      # GPU device ID
    "cache": "disk",                   # cache data in RAM for speed
    "workers": 4,                     # parallel data loader workers
    "amp": True,                      # automatic mixed precision (faster training)
    "mosaic": 0.0,               # disable mosaic in last N epochs
    "save": True,                     # save checkpoints and weights
    "val": True,                      # enable validation after each epoch
    "plots": True                     # save training/validation plots
}

In [ ]:
"""
Run-specific training parameters.

These parameters can be adjusted per experiment to test different configurations,
optimizers, learning rates, or other hyperparameters.
"""

run = input("Give name for the run: ")
epochs = input("Give number of epochs:")

if len(run) < 3:
    run = "default"

try:
    epochs = int(epochs)
except:
    epochs = 10


results = model.train(
    **default_args,
    epochs=epochs,                        # number of training epochs
    optimizer="Adam",                # optimizer: AdamW, Adam, or SGD
    lr0=0.0005,                        # initial learning rate
    momentum=0.9,                     # used with optimizers like SGD or AdamW
    weight_decay=0.0001,              # regularization to prevent overfitting
    warmup_epochs=3.0,
    name=run
)

In [ ]:
model = YOLO("material_koulutus/joonas_s_bg_lr0005_v1/weights/best.pt")
metrics = model.val()

metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

In [ ]:
"""
This script reads YOLO training results from a CSV file and visualizes key evaluation metrics:
- Training and validation losses
"""

%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

csv_path = f"material_koulutus/{run}/results.csv"
df = pd.read_csv(csv_path)
df.columns = df.columns.str.strip()

display(df.head())
display(df.tail())

df[["train/box_loss", "train/cls_loss", "train/dfl_loss"]].plot(figsize=(10, 5), title="Training Losses per Epoch")
plt.xlabel("Epoch")
plt.grid()
plt.show()